In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.custom_generator_NEW import CustomGeneratorNEW
from tohu.utils import print_generated_sequence

## Custom generator without `__init__` method

The simplest way to use `CustomGenerator` is with only class attributes (without `__init__` method).

In [2]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    cc = FakerGenerator(method="name")

In [3]:
g = QuuxGenerator()

2020-04-21 08:33:03 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [4]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


Note that while the _instance_ `g` contains attributes with the same names `"aa"` and `"bb"` as the parent class `QuuxGenerator`, internally the instance attributes are actually spawned versions of the class attributes (in particular, they are not identical):

In [5]:
print(QuuxGenerator.aa)
print(g.aa)
assert QuuxGenerator.aa is not g.aa

print(QuuxGenerator.bb)
print(g.bb)
assert QuuxGenerator.bb is not g.bb

print(QuuxGenerator.cc)
print(g.cc)
assert QuuxGenerator.cc is not g.cc

<Integer (id=878c35)>
<Integer (id=c9336b)>
<HashDigest (id=0f5148)>
<HashDigest (id=c6d7cf)>
<FakerGenerator (id=4c5f31)>
<FakerGenerator (id=3cd6bc)>


## Custom generator with `__init__` method

Tohu generators that are defined as instance attributes within the `__init__` method are also picked up.

In [6]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    
    def __init__(self, method):
        self.cc = FakerGenerator(method=method)

In [7]:
g = QuuxGenerator(method="first_name")

2020-04-21 08:33:04 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [8]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Mary')
Quux(aa=171, bb='54596E', cc='Jennifer')
Quux(aa=142, bb='2A16D0', cc='Christopher')
Quux(aa=140, bb='FDCDD3', cc='Joseph')
Quux(aa=121, bb='BDE283', cc='Kelly')


---

## Looped custom generator

In [9]:
from tohu.foreach_NEW import foreach_NEW

In [10]:
@foreach_NEW(xx=["AAA", "BBB"])
@foreach_NEW(yy=[111, 222, 333])
class QuuxGenerator(CustomGeneratorNEW):
    aa = xx
    bb = yy
    cc = Integer(100, 200)
    dd = HashDigest(length=6)

In [11]:
g = QuuxGenerator()

2020-04-21 08:33:04 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [12]:
g._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=8054ba)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=e64db6)>}

In [13]:
g._loop_runner.loop_variables["xx"].reset_loop_variable()
print_generated_sequence(g, num=5, seed=11111, sep="\n")
g._loop_runner.loop_variables["xx"].advance()
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa='AAA', bb=111, cc=163, dd='7551AA')
Quux(aa='AAA', bb=111, cc=171, dd='54596E')
Quux(aa='AAA', bb=111, cc=142, dd='2A16D0')
Quux(aa='AAA', bb=111, cc=140, dd='FDCDD3')
Quux(aa='AAA', bb=111, cc=121, dd='BDE283')
Generated sequence:

Quux(aa='BBB', bb=111, cc=163, dd='7551AA')
Quux(aa='BBB', bb=111, cc=171, dd='54596E')
Quux(aa='BBB', bb=111, cc=142, dd='2A16D0')
Quux(aa='BBB', bb=111, cc=140, dd='FDCDD3')
Quux(aa='BBB', bb=111, cc=121, dd='BDE283')


In [15]:
#g._loop_runner.loop_variables["xx"].advance()

Both the parent class `QuuxGenerator` and its instance `g` contain loop runners containing loop variables `xx` and `yy`.

In [16]:
QuuxGenerator._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=9a55b0)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=8e77c4)>}

In [17]:
g._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=8054ba)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='BBB' (tohu_id=e64db6)>}

Currently, the loop variables in `g._loop_runner` are clones of the ones in `QuuxGenerator._loop_runner`.

**TODO:** This doesn't seem necessary, and we should probably change it?!

In [20]:
assert g._loop_runner.loop_variables["xx"].is_clone_of(QuuxGenerator._loop_runner.loop_variables["xx"])
assert g._loop_runner.loop_variables["yy"].is_clone_of(QuuxGenerator._loop_runner.loop_variables["yy"])